## Reactive Functions & Expressions

In the [Dependencies and Watchers](Dependencies_and_Watchers.ipynb) guide we discovered how to express dependencies and write callback functions that are invoked when parameter values change. This low-level, imperative style of expressing dynamic behavior is powerful, and works well for capturing complex behaviors in self-contained Parameterized classes with methods and Parameters. But even if you are not primarily designing a hierarchy of classes, it is still useful to be able to express dependencies between values and computations. Param 2.0 introduces a new set of declarative dynamic computation primitives that are useful even for single expressions, letting you write simple reactive functions and expressions that are re-evaluated automatically when required.

The reactive programming model is one you might be familiar with from spreadsheets like Excel, where formulas can reference cells or ranges and dynamically (or more precisely, _reactively_) recompute when the inputs to a formula changes. In Param, Parameter objects correspond to a spreadsheet cell formula's inputs or references, and reactive expressions correspond to the formula itself. `param.bind` also allows the creation of a reactive function with arbitrary inputs.

This user guide is structured as two main sections:

- [Reactive Functions](#Reactive-Functions): Using `param.bind` to declare functions that react when their inputs change.
- [Reactive Expressions](#Reactive-Expressions): Using `param.reactive` (or `.rx()` on Parameter objects) to wrap ordinary objects and/or parameters in a proxy that acts like the underlying object but reacts when an input changes.

## Getting started

Before we dive in to discover how this works behind the scenes, let's get started with a concrete example. What reactive expressions are great for is writing pipelines of operations in a natural form, i.e., without having to explicitly track and update state or the control flow. In most cases, you can simply write the same non-reactive Python code you always write, but then use arguments and option values that are reactive, so that the pipeline will re-run if the value changes.

For an example, let's load some data into a [Pandas](https://pandas.pydata.org) DataFrame and make it reactive:

In [ ]:
import pandas as pd
import param
import param.ipython

from param import rx

In [ ]:
URL = 'https://datasets.holoviz.org/penguins/v1/penguins.csv'
df = rx(pd.read_csv(URL))
df.head(2)

Here, this is just the same code you'd normally use to make a DataFrame, apart from using `rx()` to make the DataFrame into a reactive expression. As you can see, the reactive DataFrame works like any other DataFrame, using `.head()` and any other DataFrame methods as usual. But now, let's make the fixed number `2` above into a reactive expression, and see what happens:

In [ ]:
nrows = rx(2)
df.head(nrows)

So far, nothing's changed. But what if we change the value of `nrows`?

In [ ]:
nrows.rx.set_input(4);

Whoa! As long as you are running a Jupyter notebook with a live Python process, you should have seen the dataframe "head" output _in_ _the_ _previous_ _cell_ update to the new value of `nrows`.  That's because the reactive `df` expression in that cell captures the full pipeline of operations, automatically re-running `head` because the `nrows` has now changed. 

We've done this without having to write any special callbacks or any new functions, instead using special Python objects that capture the operations you've invoked and replay them as needed when inputs change.

These updates should happen immediately (not only when the code cell finishes executing):

In [ ]:
import time

for i in range(4,9):
    nrows.rx.set_input(i)
    time.sleep(1)

You should see the previous `df.head` output react to each time `nrows` is changed, updating to reflect the current state.

We can get more complicated if we want, with a much more complex pipeline, but still matching the same code you'd write for a non-reactive Pandas DataFrame:

In [ ]:
import numpy as np

style = rx('color: white; background-color: {color}')
color = rx('darkblue')

def highlight_max(s, props=''):
    if s.dtype.kind not in 'f':
        return np.full_like(s, False)
    return np.where(s == np.nanmax(s.values), props, '')

styled_df = df.head(nrows).style.apply(highlight_max, props=style.format(color=color), axis=0)

styled_df

Here we've made two additional reactive values (`style` and `color`), and written a Pandas pipeline that uses those values using precisely the same syntax you would with a regular Pandas expression. Since this is now a reactive Pandas expression, it will re-run whenever any of those changes. To see, try executing each of the following commands, one by one:

In [ ]:
color.rx.set_input('red');

In [ ]:
nrows.rx.set_input(nrows.rx.resolve()+2);

In [ ]:
color.rx.set_input('darkblue');

In the code above, we made reactive strings, numbers, and DataFrame expressions. You can also make functions reactive, which lets you make the URL reactive as well:

In [ ]:
url = rx(URL)
df = rx(pd.read_csv)(url)
df.head(2)

In [ ]:
url.rx.set_input('https://datasets.holoviz.org/gapminders/v1/gapminders.csv');

In [ ]:
url.rx.set_input(URL);

Here, `df` is no longer wrapping up a specific DataFrame and capturing operations on it, it's wrapping up the `read_csv` call that generates the DataFrame originally, but it all still works just the same. As you can see, reactive expressions let you write code just as you usually would, but then separately control all the reactive elements of it. 

Note that we have been using Jupyter Notebook cells as a way to change these reactive values, but if you imagine using a widgets from ipywidgets or [Panel](https://panel.holoviz.org) instead, you can see how easy it is to create a reactive computation or application with user-controllable options.

## Reactive Functions

Ok, now that you've seen reactive expressions in action, let's dive into how this all works. We'll first write a simple, non-reactive function to add two arguments:

In [ ]:
def add(a, b):
    print(f'adding {a=} and {b=}')
    return a + b

add(3, 7)

Now, let's make a `Parameterized` class with some `Parameters`, here named `a` and `b`, that we'll want to add together reactively. We will also import `param.ipython` to install support for displaying "live" reactive components in an IPython or Jupyter notebook, re-rendering themselves when the inputs change.

In [ ]:
import param
import param.ipython

class Parameters(param.Parameterized):
    
    a = param.Number(1)

    b = param.Number(0)

    run = param.Event()
    
p = Parameters()

Ok, now we can use `param.bind` to "bind" parameters `a` and `b` to create a reactive function:

In [ ]:
reactive_add = param.bind(add, p.param.a, p.param.b)

reactive_add

As you can see, `reactive_add` works just like `add`, in that it adds two arguments, but in this case it's taking the value of the `a` and `b` Parameters of `p`. Parameter `a` has been "bound" to the first argument and `b` to the second, and if either of them changes, the result changes. So if we change `p.a` to 5, the output above changes immediately (it "reacts").

In [ ]:
p.a += 4

We can also call the reactive function explicitly to return the current result as a concrete, no longer reactive value:

In [ ]:
reactive_add()

The difference between `reactive_add` and `reactive_add()` is that the first one is a function, whose display will automatically update in IPython/Jupyter thanks to the extension loaded above, while the second is a specific number (the result of calling that function a single time, never to be updated further):

In [ ]:
print(type(reactive_add), type(reactive_add()))

`param.bind` follows the semantics of Python's `functools.partial`, and so if you only partially bind the required arguments, you'll get a function of the remaining arguments:

In [ ]:
add_b = param.bind(add, p.param.a)
add_b

In [ ]:
add_b(5)

Note that you can bind any accepted type to make a reactive function, not just Parameters, but static values won't trigger reactive updates (here 38 will always be the same value, while the result will depend on the current value of `p.param.a`).

In [ ]:
param.bind(add, p.param.a, b=38)

## Reactive Expressions

While reactive functions are very useful and allow writing arbitrarily complex logic, they still require writing a Python function definition, which can be verbose to write and difficult to read. With a `reactive` expression instead of an explicitly defined function, you can wrap any object or parameter value and apply operations on it, just as if you are working with the actual object, but now with reactive outputs. In other words, the reactive expression acts as a proxy for the underlying value, while supporting (almost) all operations that can be performed with the original object.

### Using Parameters

As an example, let's create reactive proxies for the `a` and `b` parameters and add them together:

In [ ]:
q = Parameters()

expr = q.param.a.rx() + q.param.b.rx() + 3

expr

The resulting reactive expression now reflects the result of this operation and will update automatically when one of the inputs to the operation changes, e.g. if we update parameter `a`:

In [ ]:
q.a += 2

### Resolving the expression

Reactive objects generally just provide whatever API the underlying object has, but there are a few extra reactive-specific methods also provided. In order to avoid any clashes between the namespace of the reactive expression and the object it is wrapping, the extra methods are in a special namespace called `.rx`.

For instance, to resolve the current value of the expression into the current value as a static (non-reactive) object, we can call `.rx.resolve()`:

In [ ]:
expr.rx.resolve()

The number displayed above is no longer tied to the reactive expression, it is its concrete output value, and so it will not update when `a` or `b` changes:

In [ ]:
q.b += 2

### Using literal objects as inputs

The convenient `param.reactive` function lets you make just about _anything_ reactive, without having to first define a new Parameterized object with explicit Parameters. E.g. we can create a `reactive` object from a static, literal value, such as a string:

In [ ]:
string_template = rx('Hello {name}!')

string_template

The `reactive` object now acts like a string so we can use the `.format` method to fill in the `string_template` with another reactive expression:

In [ ]:
name = rx('world')

str_expr = string_template.format(name=name)

str_expr

### Setting the input value

To update the input to a `reactive` object we can use the `.rx.set_input(new)` method:

In [ ]:
name.rx.set_input('there')

str_expr.rx.resolve()

### Functions

In some cases you might not have a concrete value as a starting point and start with a function instead. A fully bound function can be converted to a reactive expression letting you work with the output of a function as if it was a concrete value:

In [ ]:
param.bind(add, p.param.a, p.param.b).rx() / 2

### Special Methods & Limitations

A `reactive` proxy behaves like the underlying object it is wrapping, but only to the extent that Python allows. 
Certain operations cannot be implemented in this way, e.g. Python will not allow the `len` operation to return anything but a integer and the `is` statement always checks the immediate identity of its two operands. Reactive doesn't support operator keywords (i.e. `and`, `or`, `not`, `in` and `is`), control flow keywords (i.e. `if`, `elif`, `else`), ternary conditional expressions (i.e. `a if condition else b`), and only supports iteration keywords to a limited extent (i.e. `for` or `while`). Although it does implement an iterator interface this will only work well for fixed length collections.

#### Special methods

Therefore `reactive` implements certain operations as special methods that exist on the `.rx` namespace alongside `rx.set` and `rx.resolve`:

- `.rx.bool()`: Reactive version of `bool()`, casting the output value to a Boolean.
- `.rx.in_()`: Reactive version of `in`, testing if value is in the provided collection.
- `.rx.is_()`: Reactive version of `is`, testings the object identity against another object.
- `.rx.is_not()`: Reactive version of `is not`, testing the absence of object identity with another object.
- `.rx.len()`: Reactive version of `len()`, returning the length of the expression
- `.rx.pipe()`: Applies the given function (with static or reactive arguments) to this object.
- `.rx.when()`: Generates a new expression that only updates when the provided dependency updates.
- `.rx.where()`: Returns either the first or the second argument, depending on the current value of the expression.

#### `.rx.bool()`

Casts the current value to a Boolean True or False value:

In [ ]:
rx(1).rx.bool()

#### `.rx.in_(arg)`

Checks if current value is `.in_` the other collection

In [ ]:
rx(2).rx.in_([1, 2, 3])

#### `.rx.is_(arg)`

Checks the identity of the current value is the same as the argument to `.is_`

In [ ]:
rx(None).rx.is_(None)

#### `.rx.is_not(arg)`

Checks the identity of the current value is not the same as the argument to `.is_not`

In [ ]:
rx(None).rx.is_not(None)

#### `.rx.len()`

Returns the length of the object

In [ ]:
rx([1, 2, 3]).rx.len()

#### `.rx.pipe(func, *args, **kwargs)`

Pipes the current value into a function as the first argument, passing in additional positional and keyword arguments if provided:

In [ ]:
rx(1).rx.pipe(add, 2)

In [ ]:
rx(8).rx.pipe(str)

#### `.rx.when(*conditions)`

Useful when creating UIs to declare that the expression should only update when some other parameter changes, e.g. when a user clicks a button or triggers an expensive operation through some other mechanism.

For instance, let's say we have some expensive function (here simulated using `time.sleep`). First we bind parameters `a` and `b` to this function and create a reactive expression from this function.

In [ ]:
import time

def expensive_function(a, b):
    print(f'multiplying {a=} and {b=}')
    time.sleep(2)
    return a * b

p = Parameters()

expensive_expr = param.bind(expensive_function, p.param.a, p.param.b).rx()

The problem we face is that if we use this `expensive_expr` whenever `a` **or** `b` are changed, then the expensive computation gets triggered *twice* if we want to change both `a` _and_ `b`. We could use `p.param.update()` to change them in bulk in this particular case, but since that's not always sufficient, here we'll gate the computation behind a third variable, e.g. the `run` `Event` parameter on the `Parameters` class.

In [ ]:
gated_expr = expensive_expr.rx.when(p.param.run)

gated_expr

We can now safely change variables `a` and `b` separately without triggering the computation:

In [ ]:
p.a = 2
p.b = 4

gated_expr.rx.resolve()

But when we trigger the `run` parameter the expression will re-compute:

In [ ]:
p.param.trigger('run')

gated_expr.rx.resolve()

#### `.rx.where(x, y)`

Allows writing ternary conditions using the reactive paradigm. Ordinarily you can write something like this:

```python
value_a if some_condition else value_b
``` 

to return `value_a` or `value_b` depending on some condition. However, Python does not allow overriding `if`, so `if`  statements are not re-evaluated when the condition changes, and instead we have to rewrite this case using `where`.

First we will declare a condition, which here is simply a `reactive` expression wrapping a Boolean value:

In [ ]:
condition = rx(True)

Now let's say we want to return either Parameter `a` or `b` depending on whether the condition is True or False. We can simply pass the values to `.where()`:

In [ ]:
p = Parameters(a=1, b=2)

ternary_expr = condition.rx.where(p.param.a, p.param.b)

ternary_expr

Since the initial value is `True` it returns the current value of `a`, which is `1`. However when we set the value to `False` it will return the value of `b`:


In [ ]:
condition.rx.set_input(False)

ternary_expr.rx.resolve()

Importantly, if we now change `b` the result will be reflected by the expression, reactively unless we explicitly resolve the result:

In [ ]:
p.b = 5

ternary_expr.rx.resolve()

Here the expression value depends only on `b` thanks to the `where` condition, and thus changes to `a` will no longer trigger any downstream updates until the condition is reversed again.